### Hundrastgårdar DogParks
* se GITHUB [salgo60/Dogpark_Sweden](https://github.com/salgo60/Dogpark_Sweden)
   * [Issue 225](https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/225)
* denna Notebook [225_SAT_Dogpark.ipynb](sss)

In [1]:
import time
import datetime  
start_time = time.time()
start_str = datetime.datetime.now().strftime("%Y-%m-%d %H:%M")
print(f"Started: {start_str}")


Started: 2025-10-07 08:40


In [5]:
import requests, json, time
from bs4 import BeautifulSoup

BASE_URL = "https://www.hundlistan.se/"
params = {
    "mylisting-ajax": "1",
    "action": "get_listings",
    "security": "5e528da4d5",
    "form_data[preserve_page]": "true",
    "form_data[sort]": "latest",
    "form_data[region]": "",
    "form_data[search_location]": "",
    "form_data[lat]": "false",
    "form_data[lng]": "false",
    "listing_type": "hundpark",
    "listing_wrap": "col-md-12 grid-item"
}

features = []

for page in range(1, 40):
    params["form_data[page]"] = str(page)
    print(f"Hämtar sida {page}/39 ...", end="\r")

    r = requests.get(BASE_URL, params=params)
    if r.status_code != 200:
        print(f"\n⚠️ Sida {page} misslyckades ({r.status_code})")
        continue

    data = r.json()
    soup = BeautifulSoup(data.get("html", ""), "html.parser")

    for item in soup.select(".lf-item-container"):
        try:
            data_id = item.get("data-id")
            title = item.select_one(".listing-preview-title").get_text(strip=True)
            href = item.select_one("a")["href"]
            lan = item.select_one(".lf-contact li").get_text(strip=True)
            loc = json.loads(item.get("data-locations"))[0]
            address = loc.get("address")
            lat, lng = float(loc.get("lat")), float(loc.get("lng"))

            # --- NEW: fetch the detail page to get "Hemsida"
            hemsida_url = None
            try:
                detail = requests.get(href, timeout=10)
                if detail.status_code == 200:
                    detail_soup = BeautifulSoup(detail.text, "html.parser")
                    hemsida_tag = detail_soup.select_one(
                        "div.qla-container li a[href*='http']:has(span:contains('Hemsida'))"
                    )
                    if hemsida_tag:
                        hemsida_url = hemsida_tag["href"]
            except Exception as e:
                print(f"⚠️ Kunde inte hämta detaljer för {title}: {e}")

            popup_html = f"""
            <b>{title}</b><br>
            {address}<br>
            {lan}<br>
            <a href="{href}" target="_blank">Visa på Hundlistan</a><br>
            {f'<a href="{hemsida_url}" target="_blank">Hemsida</a>' if hemsida_url else ''}
            """.strip()
            print("Hemsida",hemsida_url)
            features.append({
                "type": "Feature",
                "geometry": {"type": "Point", "coordinates": [lng, lat]},
                "properties": {
                    "id": data_id,
                    "namn": title,
                    "adress": address,
                    "lan": lan,
                    "url": href,
                    "hemsida": hemsida_url,
                    "popup": popup_html
                }
            })

            #time.sleep(0.2)  # short delay to be polite
        except Exception as e:
            print(f"\n⚠️ Fel vid parsing: {e}")

geojson = {"type": "FeatureCollection", "features": features}

with open("hundparker_umap.geojson", "w", encoding="utf-8") as f:
    json.dump(geojson, f, ensure_ascii=False, indent=2)

print(f"\n✅ Klart! {len(features)} hundparker sparade i hundparker_umap.geojson")


Hemsida https://www.falkenberg.se/se-gora/aktiviteter/bada-simma/hundbad-i-falkenberg/
Hemsida https://www.halmstad.se/upplevaochgora/idrottmotionochfriluftsliv/friluftslivochmotion/hundrastning.n3403.html
Hemsida https://www.laholm.se/uppleva-och-gora/idrott-motion-och-friluftsliv/friluftsliv-och-motion/hundrastgard
Hemsida https://www.halmstad.se/upplevaochgora/idrottmotionochfriluftsliv/friluftslivochmotion/hundrastning.n3403.html
Hemsida https://www.laholm.se/uppleva-och-gora/idrott-motion-och-friluftsliv/friluftsliv-och-motion/hundrastgard
Hemsida https://www.halmstad.se/upplevaochgora/idrottmotionochfriluftsliv/friluftslivochmotion/hundrastning.n3403.html
Hemsida https://visitlaholm.se/uppleva-och-gora/12-kilometer-sandstrand
Hemsida https://www.halmstad.se/upplevaochgora/idrottmotionochfriluftsliv/friluftslivochmotion/hundrastning.n3403.html
Hemsida https://www.hallandsposten.se/nyheter/laholm/stort-vilthagn-saker-lekplats-for-hundar.c6995cc6-fe24-4aa4-a99c-a697e84b31d9
Hemsida 

## Extract from OSM

In [7]:
import requests

# 🗺️ Overpass API-endpoint
overpass_url = "https://overpass-api.de/api/interpreter"

# Bounding box för Sverige ungefär: (syd, väst, nord, öst)
sweden_bbox = "55.0,10.5,69.2,24.2"

# Overpass-fråga: alla noder, vägar och relationer med leisure=dog_park i Sverige
overpass_query = f"""
[out:json][timeout:60];
nwr["leisure"="dog_park"]({sweden_bbox});
out center;
"""

print("⏳ Hämtar hundrastgårdar från OSM...")
response = requests.get(overpass_url, params={"data": overpass_query})
osm_data = response.json()
print(f"✅ Hittade {len(osm_data['elements'])} objekt från OSM")



⏳ Hämtar hundrastgårdar från OSM...
✅ Hittade 912 objekt från OSM


In [46]:
# Steg 1: Samlar alla node-ID som används som byggstenar i en way/relation.
# Steg 2: Filtrerar bort dessa “interna” noder men behåller alla andra.
# Steg 3: Gör ett rent DataFrame med id, type, lat, lon och alla taggar.
import pandas as pd

# 1️⃣ Hämta alla node-ID som används i en way eller relation
used_node_ids = set()

for el in osm_data["elements"]:
    if el["type"] in ("way", "relation"):
        for ref in el.get("nodes", []):
            used_node_ids.add(ref)
        for member in el.get("members", []):
            if member.get("type") == "node":
                used_node_ids.add(member["ref"])

# 2️⃣ Behåll allt som inte är en node ELLER noder som inte används
filtered_elements = []
for el in osm_data["elements"]:
    if el["type"] != "node" or el["id"] not in used_node_ids:
        filtered_elements.append(el)

print(f"Före: {len(osm_data['elements'])}, Efter: {len(filtered_elements)}")

# 3️⃣ (valfritt) skapa DataFrame
rows = []
for el in filtered_elements:
    if el["type"] == "node":
        lat = el.get("lat")
        lon = el.get("lon")
    else:
        lat = el.get("center", {}).get("lat")
        lon = el.get("center", {}).get("lon")

    row = {
        "id": el.get("id"),
        "type": el.get("type"),
        "lat": lat,
        "lon": lon
    }
    row.update(el.get("tags", {}))
    rows.append(row)

dfOSM = pd.DataFrame(rows)
print(dfOSM.head())


Före: 912, Efter: 912
          id  type        lat        lon created_by   leisure  \
0  254731144  node  63.849216  23.119295       JOSM  dog_park   
1  852972410  node  57.268053  17.055340        NaN  dog_park   
2  861388751  node  59.457830  15.395399        NaN  dog_park   
3  871868060  node  59.365317  16.426522        NaN  dog_park   
4  874337325  node  60.053820  18.752540        NaN  dog_park   

           description                       name alt_name website  ...  \
0                  NaN                        NaN      NaN     NaN  ...   
1  Bath place for dogs                        NaN      NaN     NaN  ...   
2                  NaN       Frövi brukshundklubb      NaN     NaN  ...   
3                  NaN  Eskilstuna Brukshundklubb      NaN     NaN  ...   
4                  NaN      Haverö Brukshundklubb      NaN     NaN  ...   

  gate:type dog_park:picnic_table kartaview mapillary contact:phone  \
0       NaN                   NaN       NaN       NaN           N

In [47]:
dfOSM.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 912 entries, 0 to 911
Data columns (total 88 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     912 non-null    int64  
 1   type                   912 non-null    object 
 2   lat                    912 non-null    float64
 3   lon                    912 non-null    float64
 4   created_by             1 non-null      object 
 5   leisure                912 non-null    object 
 6   description            29 non-null     object 
 7   name                   379 non-null    object 
 8   alt_name               12 non-null     object 
 9   website                116 non-null    object 
 10  wheelchair             185 non-null    object 
 11  email                  3 non-null      object 
 12  fixme                  3 non-null      object 
 13  addr:street            32 non-null     object 
 14  access                 136 non-null    object 
 15  addr:c

### nacka kommun


In [34]:
dfHundlistan = pd.json_normalize(features)


In [36]:
dfHundlistan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 760 entries, 0 to 759
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   type                  760 non-null    object
 1   geometry.type         760 non-null    object
 2   geometry.coordinates  760 non-null    object
 3   properties.id         760 non-null    object
 4   properties.namn       760 non-null    object
 5   properties.adress     760 non-null    object
 6   properties.lan        760 non-null    object
 7   properties.url        760 non-null    object
 8   properties.hemsida    720 non-null    object
 9   properties.popup      760 non-null    object
dtypes: object(10)
memory usage: 59.5+ KB


In [29]:
# Filtrera www.nacka.se 
df[df['name'].str.contains("nacka", case=False, na=False)]

,id,type,lat,lon,created_by,leisure,description,name,alt_name,website,...,gate:type,dog_park:picnic_table,kartaview,mapillary,contact:phone,previously,seasonal,picnic_table,swimming:dog,wikipedia


In [40]:
dfHundlistanNacka = dfHundlistan[dfHundlistan['properties.hemsida'].str.contains("nacka", case=False, na=False)]  
dfHundlistanNacka

,type,geometry.type,geometry.coordinates,properties.id,properties.namn,properties.adress,properties.lan,properties.url,properties.hemsida,properties.popup
252,Feature,Point,"[18.18253, 59.2592]",listing-id-1417,Älta hundrastgård - Älta,"Almvägen (bakom ishallen, 138 30 Älta",Stockholm,https://www.hundlistan.se/listing/alta-hundras...,https://www.nacka.se/uppleva--gora/friluftsliv...,<b>Älta hundrastgård - Älta</b><br>\n ...
260,Feature,Point,"[18.27824, 59.28789]",listing-id-1394,Igelboda hundrastgård - Saltsjöbaden,"Igelbodavägen, 133 34 Saltsjöbaden",Stockholm,https://www.hundlistan.se/listing/igelboda-hun...,https://www.nacka.se/uppleva--gora/friluftsliv...,<b>Igelboda hundrastgård - Saltsjöbaden</b><br...
262,Feature,Point,"[18.25222, 59.29028]",listing-id-1395,Hundrastgård Fisksätra (Laxgatan) - Saltsjöbaden,"Laxgatan 2, 133 43 Saltsjöbaden",Stockholm,https://www.hundlistan.se/listing/hundrastgard...,https://www.nacka.se/uppleva--gora/friluftsliv...,<b>Hundrastgård Fisksätra (Laxgatan) - Saltsjö...
264,Feature,Point,"[18.26109, 59.29109]",listing-id-1396,Hundrastgård Fisksätra (Fidravägen) - Saltsjöb...,"Fidravägen 75, 133 44 Saltsjöbaden",Stockholm,https://www.hundlistan.se/listing/hundrastgard...,https://www.nacka.se/uppleva--gora/friluftsliv...,<b>Hundrastgård Fisksätra (Fidravägen) - Salts...
279,Feature,Point,"[18.23886, 59.30602]",listing-id-1393,Tollare Hundrastgård - Saltsjö-boo,"Sockenvägen 46, 132 42 Saltsjö-boo",Stockholm,https://www.hundlistan.se/listing/tollare-hund...,https://www.nacka.se/uppleva--gora/friluftsliv...,<b>Tollare Hundrastgård - Saltsjö-boo</b><br>\...
282,Feature,Point,"[18.14427, 59.3165]",listing-id-1391,Kvarnholmens Hundrastgård - Nacka,"Kvarnholmsvägen 114, 131 75 Nacka",Stockholm,https://www.hundlistan.se/listing/kvarnholmens...,https://www.nacka.se/uppleva--gora/friluftsliv...,<b>Kvarnholmens Hundrastgård - Nacka</b><br>\n...
284,Feature,Point,"[18.25358, 59.33419]",listing-id-1392,Hundrastgård Orminge - Saltsjö-boo,"Skarpövägen, 132 33 Saltsjö-boo",Stockholm,https://www.hundlistan.se/listing/hundrastgard...,https://www.nacka.se/uppleva--gora/friluftsliv...,<b>Hundrastgård Orminge - Saltsjö-boo</b><br>\...
286,Feature,Point,"[18.23886, 59.30602]",listing-id-1393,Tollare Hundrastgård - Saltsjö-boo,"Sockenvägen 46, 132 42 Saltsjö-boo",Stockholm,https://www.hundlistan.se/listing/tollare-hund...,https://www.nacka.se/uppleva--gora/friluftsliv...,<b>Tollare Hundrastgård - Saltsjö-boo</b><br>\...
294,Feature,Point,"[18.11813, 59.31144]",listing-id-1387,Henriksdalsbergets hundrastgård - Nacka,"Henriksdalsringen 1, 131 32 Nacka",Stockholm,https://www.hundlistan.se/listing/henriksdalsb...,https://www.nacka.se/uppleva--gora/friluftsliv...,<b>Henriksdalsbergets hundrastgård - Nacka</b>...
296,Feature,Point,"[18.18334, 59.30552]",listing-id-1388,Hundrastgård Långsjön - Nacka,"Vattenverksvägen 9, 131 41 Nacka",Stockholm,https://www.hundlistan.se/listing/hundrastgard...,https://www.nacka.se/uppleva--gora/friluftsliv...,<b>Hundrastgård Långsjön - Nacka</b><br>\n ...


In [31]:
# Ungefärlig bounding box för Nacka kommun
lat_min, lat_max = 59.25, 59.38
lon_min, lon_max = 18.05, 18.30

df_nacka = df[
    (df['lat'] >= lat_min) &
    (df['lat'] <= lat_max) &
    (df['lon'] >= lon_min) &
    (df['lon'] <= lon_max)
]

In [32]:
df_nacka

,id,type,lat,lon,created_by,leisure,description,name,alt_name,website,...,gate:type,dog_park:picnic_table,kartaview,mapillary,contact:phone,previously,seasonal,picnic_table,swimming:dog,wikipedia
27,6093857144,node,59.253233,18.111650,NaN,dog_park,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32,6385531312,node,59.317615,18.177065,NaN,dog_park,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70,12031529969,node,59.278929,18.134809,NaN,dog_park,NaN,Hundrastgård,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73,12133038473,node,59.334172,18.253649,NaN,dog_park,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99,41201117,way,59.265652,18.064132,NaN,dog_park,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
106,58082448,way,59.315673,18.073670,NaN,dog_park,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
107,61797138,way,59.354965,18.151858,NaN,dog_park,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
120,87242823,way,59.339033,18.070523,NaN,dog_park,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
154,164359635,way,59.311635,18.091788,NaN,dog_park,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
184,235103228,way,59.370778,18.221088,NaN,dog_park,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
import pandas as pd

# Exempelstruktur (din dfHundlistanNacka kan redan vara laddad)
# dfHundlistanNacka = pd.DataFrame({
#     "properties.namn": ["Hundbad Sickla", "Hundpark Älta"],
#     "geometry.coordinates": [[18.1492, 59.3071], [18.2275, 59.2654]],
#     "properties.url": ["https://example.com/hundbad-sickla", "https://example.com/hundpark-alta"],
#     "properties.hemsida": ["https://nacka.se", "https://nacka.se"]
# })

markdown_lines = []
markdown_lines.append("| ✅ | Namn | Koordinat | Länk Hundlistan | Länk OSM | Källa |")
markdown_lines.append("|----|-------|------------|------------------|-----------|--------|")

for _, row in dfHundlistanNacka.iterrows():
    namn = row["properties.namn"]
    lon, lat = row["geometry.coordinates"]
    hund_url = row["properties.url"]
    hemsida = row["properties.hemsida"]
    osm_link = f"https://www.openstreetmap.org/?mlat={lat}&mlon={lon}#map=16/{lat}/{lon}"

    markdown_lines.append(
        f"| | {namn} | `{lat:.5f}, {lon:.5f}` | "
        f"[Länk Hundlistan]({hund_url}) | [OSM]({osm_link}) | [Källa]({hemsida}) |"
    )

markdown = "\n".join(markdown_lines)
print(markdown)


| ✅ | Namn | Koordinat | Länk Hundlistan | Länk OSM | Källa |
|----|-------|------------|------------------|-----------|--------|
| | Älta hundrastgård - Älta | `59.25920, 18.18253` | [Länk Hundlistan](https://www.hundlistan.se/listing/alta-hundrastgard/) | [OSM](https://www.openstreetmap.org/?mlat=59.2592&mlon=18.18253#map=16/59.2592/18.18253) | [Källa](https://www.nacka.se/uppleva--gora/friluftsliv-motion/hundar-i-nacka/hundrastgardar/) |
| | Igelboda hundrastgård - Saltsjöbaden | `59.28789, 18.27824` | [Länk Hundlistan](https://www.hundlistan.se/listing/igelboda-hundrastgard/) | [OSM](https://www.openstreetmap.org/?mlat=59.28789&mlon=18.27824#map=16/59.28789/18.27824) | [Källa](https://www.nacka.se/uppleva--gora/friluftsliv-motion/hundar-i-nacka/hundrastgardar/) |
| | Hundrastgård Fisksätra (Laxgatan) - Saltsjöbaden | `59.29028, 18.25222` | [Länk Hundlistan](https://www.hundlistan.se/listing/hundrastgard-fisksatra-laxgatan/) | [OSM](https://www.openstreetmap.org/?mlat=59.29028&mlon=

In [44]:
dfHundlistan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 760 entries, 0 to 759
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   type                  760 non-null    object
 1   geometry.type         760 non-null    object
 2   geometry.coordinates  760 non-null    object
 3   properties.id         760 non-null    object
 4   properties.namn       760 non-null    object
 5   properties.adress     760 non-null    object
 6   properties.lan        760 non-null    object
 7   properties.url        760 non-null    object
 8   properties.hemsida    720 non-null    object
 9   properties.popup      760 non-null    object
dtypes: object(10)
memory usage: 59.5+ KB


In [48]:
dfOSM.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 912 entries, 0 to 911
Data columns (total 88 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     912 non-null    int64  
 1   type                   912 non-null    object 
 2   lat                    912 non-null    float64
 3   lon                    912 non-null    float64
 4   created_by             1 non-null      object 
 5   leisure                912 non-null    object 
 6   description            29 non-null     object 
 7   name                   379 non-null    object 
 8   alt_name               12 non-null     object 
 9   website                116 non-null    object 
 10  wheelchair             185 non-null    object 
 11  email                  3 non-null      object 
 12  fixme                  3 non-null      object 
 13  addr:street            32 non-null     object 
 14  access                 136 non-null    object 
 15  addr:c

In [50]:
import folium
import json
import pandas as pd
from datetime import datetime

# --------------------------------------------------
# 🗺️ 1. Grundkarta (centrerad över Sverige)
# --------------------------------------------------
m = folium.Map(location=[62.0, 16.0], zoom_start=5, tiles="OpenStreetMap")

# --------------------------------------------------
# 🐾 2. Hundlistan (dfHundlistan)
# --------------------------------------------------
hund_layer = folium.FeatureGroup(name="🐾 Hundparker (Hundlistan)")

for _, row in dfHundlistan.iterrows():
    coords = row["geometry.coordinates"]
    if not isinstance(coords, (list, tuple)) or len(coords) != 2:
        continue
    lon, lat = coords
    namn = row["properties.namn"]
    adress = row.get("properties.adress", "Okänd adress")
    lan = row.get("properties.lan", "Okänt län")
    url = row.get("properties.url", "")

    popup_html = f"""
    <div style="font-family: Arial; font-size: 13px;">
      <b style="font-size:14px; color:#2b7a78;">🐾 {namn}</b><br>
      <span style="color:#555;">📍 {adress}</span><br>
      <span style="color:#555;">🏙️ {lan}</span><br>
      <span style="color:#777;">🧭 {lat:.5f}, {lon:.5f}</span><br><br>
      <a href="{url}" target="_blank" style="color:#0077cc;">🔗 Visa på Hundlistan</a><br>
      <a href="https://www.openstreetmap.org/#map=17/{lat:.5f}/{lon:.5f}&layers=N"
         target="_blank" style="color:#d97706;">🗺️ Skapa OSM Note här</a>
    </div>
    """

    folium.Marker(
        [lat, lon],
        popup=folium.Popup(popup_html, max_width=280),
        tooltip=namn,
        icon=folium.Icon(color="green", icon="paw", prefix="fa")
    ).add_to(hund_layer)

hund_layer.add_to(m)

# --------------------------------------------------
# 🐕 3. OSM-hundrastgårdar (dfOSM)
# --------------------------------------------------
osm_layer = folium.FeatureGroup(name="🐶 Hundrastgårdar (OSM)")

for _, row in dfOSM.iterrows():
    lat, lon = row["lat"], row["lon"]
    if pd.isna(lat) or pd.isna(lon):
        continue

    name = row.get("name", "Okänd rastgård")
    osm_id = row["id"]
    osm_type = row["type"]

    popup_html = f"""
    <div style="font-family: Arial; font-size: 13px;">
      <b style="font-size:14px; color:purple;">🐕 {name}</b><br>
      <span style="color:#777;">🧭 {lat:.5f}, {lon:.5f}</span><br>
      <details>
        <summary style="color:#0077cc; cursor:pointer;">📋 Visa mer info</summary>
        <p>
          <b>OSM-ID:</b> {osm_id}<br>
          <a href="https://www.openstreetmap.org/{osm_type}/{osm_id}" 
             target="_blank" style="color:#0077cc;">🔗 Öppna i OSM</a>
        </p>
      </details>
    </div>
    """

    folium.Marker(
        [lat, lon],
        popup=folium.Popup(popup_html, max_width=280),
        tooltip=name,
        icon=folium.Icon(color="purple", icon="paw", prefix="fa")
    ).add_to(osm_layer)

osm_layer.add_to(m)

# --------------------------------------------------
# 🥾 4. SAT-trail (från SAT_full.geojson)
# --------------------------------------------------
sat_layer = folium.FeatureGroup(name="🥾 Stockholm Archipelago Trail")

with open("SAT_full.geojson", "r", encoding="utf-8") as f:
    sat_geo = json.load(f)

folium.GeoJson(
    sat_geo,
    name="SAT Trail",
    tooltip=folium.GeoJsonTooltip(fields=["name"] if "name" in sat_geo["features"][0]["properties"] else []),
    style_function=lambda x: {
        "color": "#ff6600",
        "weight": 3,
        "opacity": 0.8
    }
).add_to(sat_layer)

sat_layer.add_to(m)

# --------------------------------------------------
# 📦 5. Info-box (collapsible)
# --------------------------------------------------
created = datetime.now().strftime("%Y-%m-%d kl. %H:%M")

info_html = f"""
<style>
#infoBox {{
  position: fixed;
  top: 10px;
  left: 10px;
  z-index: 9999;
  background-color: rgba(255,255,255,0.97);
  padding: 10px 12px;
  border-radius: 12px;
  box-shadow: 0 0 10px rgba(0,0,0,0.3);
  font-family: Arial, sans-serif;
  font-size: 13px;
  max-width: 260px;
}}
#toggleBtn {{
  background-color: #0077cc;
  color: white;
  border: none;
  border-radius: 8px;
  padding: 6px 10px;
  cursor: pointer;
  font-size: 13px;
  width: 100%;
}}
#infoContent {{ display: block; margin-top: 8px; }}
</style>

<div id="infoBox">
  <button id="toggleBtn">❌ Dölj info</button>
  <div id="infoContent">
    <div style="text-align:center;">
      <img src="https://raw.githack.com/salgo60/Dogpark_Sweden/main/DogparkSweden_2.jpg"
           width="180" style="border-radius:8px; margin-bottom:5px;"><br>
      <a href="https://github.com/salgo60/Dogpark_Sweden" target="_blank"
         style="color:#0077cc; font-weight:bold;">🐾 GitHub-projektet</a>
    </div>
    <hr>
    <b>📅 Skapad:</b> {created}<br>
    <b>📍 Lager:</b>
    <ul style="margin-top:4px; padding-left:18px;">
      <li>🐾 Hundlistan (grön)</li>
      <li>🐶 OSM (lila)</li>
      <li>🥾 SAT-trail (orange)</li>
    </ul>
  </div>
</div>

<script>
const btn = document.getElementById('toggleBtn');
const content = document.getElementById('infoContent');
let open = true;
btn.onclick = () => {{
  open = !open;
  content.style.display = open ? 'block' : 'none';
  btn.innerText = open ? '❌ Dölj info' : '📖 Visa info';
}};
</script>
"""
m.get_root().html.add_child(folium.Element(info_html))

# --------------------------------------------------
# 📍 6. “Min plats”-knapp (geolocation)
# --------------------------------------------------
geolocate_js = """
<script>
function addLocateButton(map){
  const btn = L.control({position: 'bottomright'});
  btn.onAdd = function(){
    const div = L.DomUtil.create('div', 'leaflet-bar leaflet-control leaflet-control-custom');
    div.innerHTML = '📍<br><small style="font-size:10px;">Min plats</small>';
    div.style.backgroundColor = 'white';
    div.style.cursor = 'pointer';
    div.style.textAlign = 'center';
    div.style.padding = '4px';
    div.title = 'Visa min plats';
    div.onclick = function(){
      if (navigator.geolocation){
        navigator.geolocation.getCurrentPosition(
          function(pos){
            const lat = pos.coords.latitude;
            const lon = pos.coords.longitude;
            const userMarker = L.marker([lat, lon], {
              icon: L.icon({
                iconUrl: 'https://cdn-icons-png.flaticon.com/512/64/64113.png',
                iconSize: [24, 24]
              })
            }).addTo(map);
            userMarker.bindPopup('📍 Du är här!<br><b>Lat:</b> '+lat.toFixed(5)+'<br><b>Lon:</b> '+lon.toFixed(5)).openPopup();
            map.setView([lat, lon], 14);
          },
          function(err){ alert('Kunde inte hämta position: ' + err.message); }
        );
      } else {
        alert('Geolocation stöds inte av din webbläsare.');
      }
    };
    return div;
  };
  btn.addTo(map);
}
setTimeout(function() {
  for (const key in window) {
    if (key.startsWith("map_")) {
      addLocateButton(window[key]);
      break;
    }
  }
}, 500);
</script>
"""
m.get_root().html.add_child(folium.Element(geolocate_js))

# --------------------------------------------------
# 🧭 7. Legend (färger och symboler)
# --------------------------------------------------
legend_html = """
<div style="
  position: fixed; 
  bottom: 30px; left: 10px; 
  z-index: 9999; 
  background: white; 
  border-radius: 10px; 
  padding: 8px 10px;
  font-family: Arial; 
  font-size: 13px;
  box-shadow: 0 0 6px rgba(0,0,0,0.3);
">
<b>🗺️ Lager</b><br>
<span style='color:green;'>🟢</span> Hundlistan<br>
<span style='color:purple;'>🟣</span> OSM-hundrastgårdar<br>
<span style='color:#ff6600;'>🟠</span> SAT-trail
</div>
"""
m.get_root().html.add_child(folium.Element(legend_html))

# --------------------------------------------------
# 🎛️ 8. Layer Control + Spara
# --------------------------------------------------
folium.LayerControl(collapsed=False).add_to(m)

outfile = "output/SAT225_hundparker.html"
m.save(outfile)
print(f"✅ Klar: {outfile}")


✅ Klar: output/SAT225_hundparker.html


In [2]:
end_time = time.time()
duration = end_time - start_time
print(f"Finished in {duration:.2f} seconds.")


Finished in 19.81 seconds.
